In [1]:
# importation des librairies
from bs4 import BeautifulSoup
import requests
import lxml
import pandas as pd

In [20]:
# fonction permettant de créer une nouvelle dataframe avec les colonnes voulues
def export_table_and_print(data):
    table = pd.DataFrame(data, columns=[
                         'Name', 'Price', 'Rating', 'Instock', 'Link'])
    table.index = table.index + 1    

# fonction qui cherche chaque attribut demandé et les insère dans un dictionnaire
def get_book_attributes(book):
    name = book.find('h3').find('a').attrs['title']
    price = book.find('p', class_='price_color').text
    price = price.replace('Â£', "")
    rating = book.find('p', class_='star-rating')
    dispo = book.find('p', class_='instock availability').text
    dispo = dispo.replace("\n\n    \n        ", "")
    dispo = dispo.replace("\n    \n", "")
    link = book.find('h3').find('a').attrs['href']
    l = rating['class']
    s = " ".join(l)
    s = s.replace("star-rating ", "")
    # remplace le texte en chiffre
    if s == "One":
        rating_number = 1
    elif s =="Two":
        rating_number = 2
    elif s =="Three":
        rating_number = 3
    elif s =="Four":
        rating_number = 4
    elif s =="Five":
        rating_number = 5
    data['Name'].append(name)
    data['Price'].append(price)
    data['Rating'].append(rating_number)
    data['Instock'].append(dispo)
    data['Link'].append(link)
    return data

In [21]:
data = {
            'Name' : [], 'Price' : [], 'Rating' : [], 'Instock' : [], 'Link' : []
        }

for i in range(1, 51):
    url = f"https://books.toscrape.com/catalogue/page-{i}.html"
    page = requests.get(url)
    # je vérifie si l'url est bonne
    if page.status_code == requests.codes.ok:
        bs = BeautifulSoup(page.text, 'lxml')
        # récupération de tous les objets
        list_all_book = bs.findAll('article', class_='product_pod')        
        
        for book in list_all_book:
            get_book_attributes(book)

# réinitialisation de l'index   
df = pd.DataFrame(data).reset_index()
df

,index,Name,Price,Rating,Instock,Link
0,0,A Light in the Attic,51.77,3,In stock,a-light-in-the-attic_1000/index.html
1,1,Tipping the Velvet,53.74,1,In stock,tipping-the-velvet_999/index.html
2,2,Soumission,50.10,1,In stock,soumission_998/index.html
3,3,Sharp Objects,47.82,4,In stock,sharp-objects_997/index.html
4,4,Sapiens: A Brief History of Humankind,54.23,5,In stock,sapiens-a-brief-history-of-humankind_996/index...
...,...,...,...,...,...,...
995,995,Alice in Wonderland (Alice's Adventures in Won...,55.53,1,In stock,alice-in-wonderland-alices-adventures-in-wonde...
996,996,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",57.06,4,In stock,ajin-demi-human-volume-1-ajin-demi-human-1_4/i...
997,997,A Spy's Devotion (The Regency Spies of London #1),16.97,5,In stock,a-spys-devotion-the-regency-spies-of-london-1_...
998,998,1st to Die (Women's Murder Club #1),53.98,1,In stock,1st-to-die-womens-murder-club-1_2/index.html


In [23]:
# transformation du dataframe en CSV
df.to_csv('csv/book.csv', index=False)